api from https://banks.data.fdic.gov/docs/#/Structure/searchInstitutions

based on work from
https://doi.org/10.1016/j.ribaf.2017.07.104 - Predicting bank failure: An improvement by implementing a machine-learning approach to classical financial ratios
https://doi.org/10.1016/j.dss.2012.11.015 - Partial Least Square Discriminant Analysis for bankruptcy prediction
https://doi.org/10.1016/j.eswa.2008.01.053 - Effects of feature construction on classification performance: An empirical study in bank failure prediction

In [1]:
import requests
import pandas as pd
from io import StringIO
from urllib.parse import quote_plus

# pd.set_option('display.max_rows', None)

## Construct API call

In [2]:
# function to get data from BankFind API
def getData(url: str, filter: str, fields: str, sortby: str = 'CERT', order: str = 'ASC', n: int = 10000, suffix: str = '&offset=0&format=csv&download=false&filename=data_file') -> pd.DataFrame:
    request = requests.get(url + 'filters=' + quote_plus(filter) + '&fields=' + quote_plus(fields) + '&sort_by=' + sortby + '&sort_order=' + order + '&limit=' + str(n) + suffix).text
    return pd.read_csv(StringIO(request))

### Get lists of all banks 2006-present

In [3]:
# strings for getting lists of active and failed banks
institutionurl = 'https://banks.data.fdic.gov/api/institutions?'
filtersfailed = 'ACTIVE:0 AND DATEUPDT:[\"2006-01-01\" TO \"2023-12-31\"]'
filtersactive = 'ACTIVE:1 AND DATEUPDT:[\"2006-01-01\" TO \"2023-12-31\"]'
bankfields = 'STALP,NAME,ACTIVE,CERT,DATEUPDT'


# get failed banks
failedbanks = getData(institutionurl, filtersfailed, bankfields)

# get active banks
activebanks = getData(institutionurl, filtersactive, bankfields)

In [5]:
failedbanks.head()

,ACTIVE,CERT,DATEUPDT,ID,NAME,STALP
0,0,9,02/27/2008,9,Union Trust Company,ME
1,0,46,01/07/2022,46,Merchants Bank of Alabama,AL
2,0,47,07/10/2020,47,Traders & Farmers Bank,AL
3,0,57,11/21/2006,57,Community Bank,AL
4,0,59,01/09/2013,59,The Citizens Bank,AL


In [6]:
activebanks.head()

,ACTIVE,CERT,DATEUPDT,ID,NAME,STALP
0,1,14,06/05/2023,14,State Street Bank and Trust Company,MA
1,1,35,09/02/2022,35,AuburnBank,AL
2,1,39,03/28/2023,39,Robertson Banking Company,AL
3,1,41,08/31/2022,41,Phenix-Girard Bank,AL
4,1,49,08/31/2022,49,Bank of Evergreen,AL


In [7]:
print(f'number of failed banks: {len(failedbanks)}')
print(f'number of active banks: {len(activebanks)}')

number of failed banks: 4943
number of active banks: 4618


### Get financials of banks

In [20]:
# financials

financialsurl = 'https://banks.data.fdic.gov/api/financials?'
# filters = ['ACTIVE:1', 'REPYEAR:[2000 TO 2023]']
filters = 'RISDATE:20211231'
featureslist =  ['NAME,RISDATE,CERT,',
                'LNATRESR,ELNLOS,NIM,EAMINTAN,LNLSGRS,NTLNLS,EQ,AVASSETJ,',
                'RBCT1,EQTOTR,EQV,LNLSNET,LIAB,LIABEQR,LIABEQ,DEP,',
                'NIMY,NIMR,NONIXR,PTAXNETINCR,ITAX,',
                'ROA,ROE,NETINC,EEFFR,CHBAL']
features = ''.join(featureslist)

featurenames = {'LNATRESR': 'LOAN LOSS RESERVE/GROSS LN&LS',
                'ELNLOS' : 'PROVISIONS FOR LN & LEASE LOSSES',
                'NIM' : 'NET INTEREST INCOME',
                'EAMINTAN' : 'AMORT & IMPAIR LOSS AST',
                'LNLSGRS' : 'LOANS AND LEASES, GROSS',
                'NTLNLS' : 'TOTAL LN&LS NET CHARGE-OFFS',
                'EQ' : 'Equity Capital',
                'AVASSETJ' : 'AVERAGE ASSETS-ADJUSTED-PCA',
                'RBCT1' : 'TIER 1 RBC-PCA',
                'EQTOTR' : 'TOTAL EQUITY CAPITAL RATIO',
                'EQV' : 'BANK EQUITY CAPITAL/ASSETS',
                'LNLSNET' : 'LOANS AND LEASES-NET',
                'CUSLI' : 'CUSTOMERS ACCEPTANCES',
                'LIAB' : 'TOTAL LIABILITIES',
                'LIABEQR' : 'TOTAL LIABILITIES & CAPITAL RATIO',
                'LIABEQ' : 'TOTAL LIABILITIES & CAPITAL',
                'DEP' : 'Total deposits',
                'NIMY' : 'NET INTEREST MARGIN',
                'NIMR' : 'NET INTEREST INCOME RATIO',
                'IOTHFEE' : 'OTHER FEE INCOME',
                'NONIXR' : 'TOTAL NONINTEREST EXPENSE RATIO',
                'PTAXNETINCR' : 'PRE-TAX NET INCOME OPERATING INCOME RATIO',
                'ITAX' : 'APPLICABLE INCOME TAXES',
                'ROA' : 'Return on assets (ROA)',
                'ROE' : 'Return on equity (ROE)',
                'NETINC' : 'Net income',
                'EEFFR' : 'EFFICIENCY RATIO',
                'CHBAL' : 'CASH & DUE FROM DEPOSITORY INST'

                }



In [21]:
activefinancials = getData(financialsurl, filters, features)

In [22]:
activefinancials.head()

,AVASSETJ,CERT,CHBAL,DEP,EAMINTAN,EEFFR,ELNLOS,EQ,EQTOTR,EQV,...,NIM,NIMR,NIMY,NONIXR,NTLNLS,PTAXNETINCR,RBCT1,RISDATE,ROA,ROE
0,290403000.0,14,109322000,260805000,243000.0,69.913938,-29000.0,27821000.0,8.943847,8.943847,...,1996000.0,0.632477,0.776841,2.573955,2000.0,1.007971,18845000.0,20211231,0.841930,9.88
1,1070188.0,35,78839,996948,0.0,70.225961,-600.0,100951.0,9.139783,9.139783,...,24035.0,2.331261,2.514126,1.911178,79.0,0.868488,100059.0,20211231,0.741522,7.52
2,410344.0,39,54893,374169,0.0,57.542931,300.0,36189.0,8.779710,8.779710,...,12091.0,3.080243,3.411668,2.381705,-8.0,1.680874,36507.0,20211231,1.570055,16.73
3,277748.0,41,49033,242418,0.0,66.720850,100.0,36064.0,12.643432,12.643432,...,6880.0,2.627860,2.780162,2.351705,41.0,1.134792,36019.0,20211231,1.678698,12.24
4,384528.0,46,28545,345793,0.0,75.833452,24.0,35665.0,9.313178,9.313178,...,11151.0,2.898039,3.075171,2.778490,45.0,0.879210,35597.0,20211231,0.692868,7.46


In [23]:
print(f'number of financial reports (banks): {len(activefinancials)}')

number of financial reports (banks): 4904


In [43]:
activedata = activebanks.merge(activefinancials, on = 'CERT', how = 'left', suffixes=['_b', '_f'])

In [44]:
activedata.head(20)

,ACTIVE,CERT,DATEUPDT,ID_b,NAME_b,STALP,AVASSETJ,CHBAL,DEP,EAMINTAN,...,NIM,NIMR,NIMY,NONIXR,NTLNLS,PTAXNETINCR,RBCT1,RISDATE,ROA,ROE
0,1,14,06/05/2023,14,State Street Bank and Trust Company,MA,290403000.0,109322000.0,260805000.0,243000.0,...,1996000.0,0.632477,0.776841,2.573955,2000.0,1.007971,18845000.0,20211231.0,0.841930,9.88
1,1,35,09/02/2022,35,AuburnBank,AL,1070188.0,78839.0,996948.0,0.0,...,24035.0,2.331261,2.514126,1.911178,79.0,0.868488,100059.0,20211231.0,0.741522,7.52
2,1,39,03/28/2023,39,Robertson Banking Company,AL,410344.0,54893.0,374169.0,0.0,...,12091.0,3.080243,3.411668,2.381705,-8.0,1.680874,36507.0,20211231.0,1.570055,16.73
3,1,41,08/31/2022,41,Phenix-Girard Bank,AL,277748.0,49033.0,242418.0,0.0,...,6880.0,2.627860,2.780162,2.351705,41.0,1.134792,36019.0,20211231.0,1.678698,12.24
4,1,49,08/31/2022,49,Bank of Evergreen,AL,75044.0,5253.0,66576.0,0.0,...,2205.0,3.077322,3.282217,2.641892,13.0,0.880631,8959.0,20211231.0,0.736883,5.88
5,1,50,08/31/2022,50,Community Spirit Bank,AL,179128.0,17183.0,161191.0,0.0,...,6569.0,3.757897,4.130662,3.164666,18.0,1.300305,17551.0,20211231.0,1.256828,12.55
6,1,51,08/31/2022,51,The Bank of Vernon,AL,221898.0,52924.0,196239.0,0.0,...,6954.0,3.191466,3.349721,2.573733,258.0,1.629236,28876.0,20211231.0,1.275852,9.98
7,1,52,08/31/2022,52,The Citizens Bank of Winfield,AL,288139.0,2971.0,179583.0,0.0,...,9568.0,3.290999,3.546147,1.576364,123.0,1.970541,55881.0,20211231.0,1.876296,8.70
8,1,54,08/31/2022,54,The Exchange Bank of Alabama,AL,368291.0,66441.0,314183.0,0.0,...,9210.0,2.584203,2.690404,2.274716,-13.0,0.899280,46193.0,20211231.0,0.729525,5.70
9,1,58,08/31/2022,58,United Bank,AL,1030052.0,107714.0,867746.0,0.0,...,30907.0,3.153468,3.509795,2.632192,-437.0,2.210703,96160.0,20211231.0,1.715649,18.88


In [45]:
activedata.describe()

,ACTIVE,CERT,ID_b,AVASSETJ,CHBAL,DEP,EAMINTAN,EEFFR,ELNLOS,EQ,...,NIM,NIMR,NIMY,NONIXR,NTLNLS,PTAXNETINCR,RBCT1,RISDATE,ROA,ROE
count,4618.0,4618.000000,4618.000000,4.587000e+03,4.596000e+03,4.596000e+03,4587.000000,4596.000000,4.587000e+03,4.587000e+03,...,4.587000e+03,4596.000000,4596.000000,4596.000000,4.587000e+03,4596.000000,4.587000e+03,4596.0,4596.000000,4587.000000
mean,1.0,20553.705717,20553.705717,4.766443e+06,7.594416e+05,4.046667e+06,813.414650,71.899824,-6.000187e+03,4.822066e+05,...,1.063144e+05,3.094271,3.315513,3.131517,5.430903e+03,1.511496,4.133411e+05,20211231.0,1.295739,10.872372
std,0.0,17169.729263,17169.729263,7.399371e+07,1.362580e+07,6.050378e+07,12080.724817,176.105740,1.701543e+05,7.034218e+06,...,1.355185e+06,1.053978,1.148447,8.410936,9.526678e+04,9.411049,5.980369e+06,0.0,7.467175,20.282036
min,1.0,14.000000,14.000000,3.006000e+03,0.000000e+00,5.000000e+02,-165.000000,0.000000,-9.068000e+06,4.710000e+02,...,-6.910000e+02,-0.347460,-0.583803,0.000000,-1.000000e+04,-52.101950,4.710000e+02,20211231.0,-52.107642,-130.270000
25%,1.0,8633.250000,8633.250000,1.393510e+05,1.244925e+04,1.218912e+05,0.000000,56.540984,0.000000e+00,1.542750e+04,...,4.136500e+03,2.649715,2.825864,2.017570,-9.000000e+00,0.832498,1.521550e+04,20211231.0,0.727860,6.665000
50%,1.0,15942.000000,15942.000000,3.121670e+05,3.264050e+04,2.730520e+05,0.000000,64.796126,7.000000e+01,3.308200e+04,...,9.256000e+03,3.061854,3.272930,2.420993,6.000000e+00,1.260362,3.230700e+04,20211231.0,1.088583,10.370000
75%,1.0,28867.250000,28867.250000,7.675140e+05,9.187600e+04,6.701580e+05,0.000000,74.733855,4.005000e+02,8.144400e+04,...,2.278000e+04,3.465980,3.706416,2.872354,1.305000e+02,1.656883,7.896300e+04,20211231.0,1.454242,13.990000
max,1.0,91325.000000,91325.000000,3.334925e+09,7.403950e+08,2.549631e+09,558000.000000,7780.769231,7.400000e+05,3.028480e+08,...,5.200600e+07,21.646414,24.048977,362.671601,3.938000e+06,615.683193,2.669100e+08,20211231.0,486.389989,1273.250000


In [46]:
activedata.isna().sum()

ACTIVE          0
CERT            0
DATEUPDT        0
ID_b            0
NAME_b          0
STALP           0
AVASSETJ       31
CHBAL          22
DEP            22
EAMINTAN       31
EEFFR          22
ELNLOS         31
EQ             31
EQTOTR         22
EQV            22
ID_f           22
ITAX           31
LIAB           22
LIABEQ         22
LIABEQR        22
LNATRESR       22
LNLSGRS        22
LNLSNET        22
NAME_f         22
NETINC         31
NIM            31
NIMR           22
NIMY           22
NONIXR         22
NTLNLS         31
PTAXNETINCR    22
RBCT1          31
RISDATE        22
ROA            22
ROE            31
dtype: int64